In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
%matplotlib inline

In [2]:
df = pd.read_csv('df.csv',index_col=0)
df.head(10)

,match_id,time,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,result
0,001a1f289e3bab22_b,0.0,2415.0,475.0,475.0,475.0,475.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,001a1f289e3bab22_b,1.0,2430.0,475.0,475.0,475.0,475.0,530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,001a1f289e3bab22_b,2.0,2751.0,532.0,552.0,532.0,532.0,603.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,001a1f289e3bab22_b,3.0,3929.0,849.0,817.0,791.0,686.0,786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,001a1f289e3bab22_b,4.0,5219.0,1099.0,1099.0,1107.0,976.0,938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
5,001a1f289e3bab22_b,5.0,6418.0,1466.0,1278.0,1352.0,1196.0,1126.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,1.5,0.0
6,001a1f289e3bab22_b,6.0,8156.0,1667.0,1708.0,1574.0,1687.0,1520.0,2.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,3.5,0.0
7,001a1f289e3bab22_b,7.0,9520.0,1970.0,2097.0,1708.0,1995.0,1750.0,2.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.00,3.5,0.0
8,001a1f289e3bab22_b,8.0,10681.0,2147.0,2414.0,1988.0,2244.0,1888.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,4.5,0.0
9,001a1f289e3bab22_b,9.0,12036.0,2605.0,2533.0,2321.0,2535.0,2042.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,4.5,0.0


In [3]:
df.shape

(286908, 25)

## Test process on smaller dataframes

In [4]:
# this df contains 2 games worth of rows
dft = df.head(76).copy()

In [5]:
# this df contains 1 game worth of rows
dfs = df.head(38).copy()

In [127]:
def game_length(df):
    """Input is a dataframe.
    Output is a list of how many rows are in each group
    """
    arr = df.groupby('match_id').size()
    gl_list = list(arr.values)
    return gl_list

def parse_game(df, gl):
    """Inputs are 1. a dataframe, 2. the length of each group provided by function 'game_length()'
    Output is a list of 3d arrays, each element contains 10 rows (which are equivalent to minutes 
       for this dataset). The function tracks the length of each group and stops at length minus 1. 
       After a group is finished the function performs the same process on the next group.
    """
    df_list = []
    
    game_length_counter = 0
    
    gb = df.groupby('match_id')
    
    for k in gb.groups.keys():
        
        t = 10
        start = 0
        g_df = gb.get_group(k)
        length = gl[game_length_counter]
        
        while t < length:
            # take a 10 row chunk and convert it to array
            data = g_df.iloc[:,2:][start:t].values
            
            re_data = data.reshape((1,10,23))
            
            df_list.append(re_data)
            
            # stop when t reaches length - 1
            start += 1
            t += 1
        game_length_counter += 1
    return df_list  

In [145]:
def parse_game(grp):
    """Inputs are 1. a dataframe, 2. the length of each group provided by function 'game_length()'
    Output is a list of 3d arrays, each element contains 10 rows (which are equivalent to minutes 
       for this dataset). The function tracks the length of each group and stops at length minus 1. 
       After a group is finished the function performs the same process on the next group.
    """  
    t = 10
    start = 0
   
    length = len(grp)

    while t < length:
        
        # take a 10 row chunk and convert it to array
        
        data = grp.iloc[:,2:][start:t].values
   
        df_list.append(data)
            
        # stop when t reaches length - 1
        start += 1
        t += 1
        
    return df_list  

In [146]:
df_list = []

gb = df.groupby('match_id')

gb.apply(lambda group: parse_game(group))

match_id
001a1f289e3bab22_b    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
001a1f289e3bab22_r    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
001f50c8547e2e73_b    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
001f50c8547e2e73_r    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
002f613ae09ad421_r    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
003aefd58000cbd6_b    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
003aefd58000cbd6_r    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
003dba1358cd918a_b    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
003dba1358cd918a_r    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
005a9b7b25ee9ac5_b    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
005a9b7b25ee9ac5_r    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
0073b31255641f6c_b    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
0073b31255641f6c_r    [[[2415.0, 475.0, 475.0, 475.0, 475.0, 515.0, ...
0076dde983f72733_b    [[[2415.0, 475.0, 475.0, 475.0, 4

In [147]:
length = len(df_list)
print(length)
array_a = np.array(df_list)
array_a = array_a.reshape((length,10,23))
array_a

211824


array([[[  2.41500000e+03,   4.75000000e+02,   4.75000000e+02, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.43000000e+03,   4.75000000e+02,   4.75000000e+02, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.75100000e+03,   5.32000000e+02,   5.52000000e+02, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  9.52000000e+03,   1.97000000e+03,   2.09700000e+03, ...,
           1.00000000e+00,   3.50000000e+00,   0.00000000e+00],
        [  1.06810000e+04,   2.14700000e+03,   2.41400000e+03, ...,
           7.50000000e-01,   4.50000000e+00,   0.00000000e+00],
        [  1.20360000e+04,   2.60500000e+03,   2.53300000e+03, ...,
           7.50000000e-01,   4.50000000e+00,   0.00000000e+00]],

       [[  2.43000000e+03,   4.75000000e+02,   4.75000000e+02, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  2.75100000e+03,   5.32000000e+02,   5.52000000e+0